In [1]:
import pandas as pd

# Original-DataFrame laden
df = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx", engine="openpyxl")


In [4]:
# Cluster_Duplikate neu erstellen
df["Cluster_Duplikate"] = -1

for idx, gruppe in enumerate(df["Duplikat_Gruppe"].dropna().unique()):
    df.loc[df["Duplikat_Gruppe"] == gruppe, "Cluster_Duplikate"] = idx


In [2]:
# Das BERT-Clustering-Resultat laden
df_bert = pd.read_excel("../../data/df_de_final_clustered_bert.xlsx", engine="openpyxl")


In [5]:
# Mapping-Tabelle erstellen
id_to_cluster = df_bert.set_index("ID_gesamt")["Cluster_HDBSCAN"].to_dict()

# Neue Spalte im originalen df
def assign_bert_cluster(row):
    if row["Cluster_Duplikate"] != -1:
        # Wenn es eine Duplikat-Gruppe gab: Behalte Cluster_Duplikate
        return row["Cluster_Duplikate"]
    else:
        # Wenn es keine Duplikat-Gruppe gab: nutze Cluster_HDBSCAN von BERT
        return id_to_cluster.get(row["ID_gesamt"], -1)  # Falls keine Zuordnung, dann -1 (Noise)

df["Cluster_Final_BERT"] = df.apply(assign_bert_cluster, axis=1)


In [6]:
cluster_counts = df["Cluster_Final_BERT"].value_counts().sort_index()
print(cluster_counts)


Cluster_Final_BERT
-1       1900
 0         59
 1         59
 2         60
 3          3
         ... 
 1310       3
 1311       3
 1312       2
 1313       3
 1314       2
Name: count, Length: 1316, dtype: int64


In [7]:
anzahl_gesamt = len(df)
anzahl_geclusterte = (df["Cluster_Final_BERT"] != -1).sum()
print(f"Anteil geclusterte Fragen: {anzahl_geclusterte} / {anzahl_gesamt} = {anzahl_geclusterte / anzahl_gesamt:.2%}")


Anteil geclusterte Fragen: 5559 / 7459 = 74.53%
